In [26]:
import os
import sys

parent_dir = os.path.abspath("/home/leandre/Projects/BioMoQA_Playground")

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from src.data_pipeline.ipbes.create_ipbes_raw import loading_pipeline_from_raw

In [27]:
datasets = loading_pipeline_from_raw(multi_label=True)

2025-07-17 16:41:55,277 - INFO - load raw positive datasets
2025-07-17 16:41:55,277 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES IAS_2352922
2025-07-17 16:41:55,278 - INFO - 128


Resolving data files:   0%|          | 0/128 [00:00<?, ?it/s]

2025-07-17 16:41:55,649 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES IAS_2352922
2025-07-17 16:41:55,650 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES SUA_2344805
2025-07-17 16:41:55,651 - INFO - 86


Resolving data files:   0%|          | 0/86 [00:00<?, ?it/s]

2025-07-17 16:41:55,785 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES SUA_2344805
2025-07-17 16:41:55,786 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES VA_2345372
2025-07-17 16:41:55,787 - INFO - 128


Resolving data files:   0%|          | 0/128 [00:00<?, ?it/s]

2025-07-17 16:41:55,930 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES VA_2345372
2025-07-17 16:41:55,930 - INFO - pos_ds features for IAS : {'DOI': Value(dtype='string', id=None), 'Title': Value(dtype='string', id=None), 'Abstract Note': Value(dtype='string', id=None), 'Language': Value(dtype='string', id=None), 'Item Type': Value(dtype='string', id=None), 'Publication Year': Value(dtype='int32', id=None)}
2025-07-17 16:41:55,931 - INFO - load raw negative dataset
2025-07-17 16:41:55,931 - INFO - creating corpus dataset


Resolving data files:   0%|          | 0/2524 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/42 [00:00<?, ?it/s]

2025-07-17 16:41:57,296 - INFO - dataset loaded
2025-07-17 16:41:57,297 - INFO - ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr']
2025-07-17 16:41:57,298 - INFO - renaming raw positive datasets
2025-07-17 16:41:57,312 - INFO - Finished positives and negatives creation pipeline


In [28]:
datasets

([Dataset({
      features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
      num_rows: 5251
  }),
  Dataset({
      features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
      num_rows: 6281
  }),
  Dataset({
      features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
      num_rows: 3109
  })],
 Dataset({
     features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
     num_rows: 4227131
 }))

In [29]:
pos_ds_list=datasets[0]
neg_ds=datasets[1]

In [30]:
clean_pos=pos_ds_list[0].filter(lambda batch : [batch['doi'][i] is None and batch['Item Type'][i] == 'journalArticle' and (batch['title'][i] is not None or batch['abstract'][i] is not None) for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)

In [31]:
print(clean_pos)

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
    num_rows: 328
})


In [32]:
clean_pos_a=pos_ds_list[0].filter(lambda batch : [batch['doi'][i] is not None and batch['Item Type'][i] == 'journalArticle' for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)

In [33]:
print(clean_pos_a)

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
    num_rows: 3829
})


In [34]:
clean_pos_b=clean_pos_a.filter(lambda batch : [(batch['abstract'][i] is not None and batch['title'][i] is not None) for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)
print(clean_pos_b)

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
    num_rows: 2253
})


In [35]:
none_abs=clean_pos_a.filter(lambda batch : [batch['abstract'][i] is None or batch['title'][i] is None for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)
print(none_abs)

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
    num_rows: 1576
})


In [36]:
print(none_abs[2])

{'doi': 'https://doi.org/10.1890/02-5002', 'title': 'Fire and grazing impacts on plant diversity and alien plant invasions in the southern Sierra Nevada', 'abstract': None, 'Item Type': 'journalArticle', 'Publication Year': 2003}


In [37]:
none_abs_instance=neg_ds.filter(lambda batch : [batch['display_name'][i] == 'Germination and early growth of Ailanthus< and tulip poplar in three levels of forest disturbance' if batch['display_name'][i] is not None else False for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)
print(none_abs_instance)

Dataset({
    features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
    num_rows: 0
})


In [38]:
pos_doi_set=set(pos_ds_list[0]['doi'])
pos_doi_set.remove(None)
none_abs_instance=neg_ds.filter(lambda batch : [any(batch['doi'][i].endswith(p_doi) for p_doi in pos_doi_set) if batch['doi'][i] is not None else False for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=30)

In [39]:
print(len(pos_doi_set))

3861


In [40]:
print(none_abs_instance)

Dataset({
    features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
    num_rows: 941
})


In [41]:
print(neg_ds['doi'][0:10])

['https://doi.org/10.1016/s0016-0032(29)90208-8', 'https://doi.org/10.1016/s0016-0032(29)90638-4', 'https://doi.org/10.1098/rstl.1829.0026', 'https://doi.org/10.1016/s0016-0032(29)90698-0', 'https://doi.org/10.1080/14786442908675166', 'https://doi.org/10.5962/bhl.title.109375', 'https://doi.org/10.1016/s0016-0032(29)90539-1', 'https://doi.org/10.1016/s0016-0032(29)90045-4', 'https://doi.org/10.1016/s0016-0032(29)90465-8', 'https://doi.org/10.1098/rstl.1816.0012']


In [42]:
print(neg_ds.filter(lambda batch : [batch['id'][i].endswith('W2142524821') if batch['doi'][i] is not None else False for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20))

Dataset({
    features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
    num_rows: 0
})


In [43]:
print(neg_ds['id'][0])

https://openalex.org/W1979751022


In [44]:
recent_pos=clean_pos_b.filter(lambda batch : [batch["Publication Year"][i]>=2021 if batch["Publication Year"][i] is not None else False for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)
print(recent_pos)

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
    num_rows: 158
})


In [ ]:
from src.data_pipeline.ipbes.preprocess_ipbes import data_pipeline

clean_ds,folds_per_run=data_pipeline(2,1,balance_coeff=1,multi_label=True)
print(clean_ds)
print(clean_ds.features)

TypeError: data_pipeline() missing 1 required positional argument: 'balance_coeff'

In [ ]:

print(clean_ds.filter(lambda batch : [batch['doi'][i] is None and (batch['IAS']==1 or batch['SUA']==1 or batch['VA']==1 ) for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20))
print(clean_ds.filter(lambda batch : [batch['abstract'][i] is None for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20))
print(clean_ds.filter(lambda batch : [batch['title'][i] is None for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20))
print(clean_ds.to_pandas()['Item Type'].unique())



Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year', 'IAS', 'SUA', 'VA'],
    num_rows: 747444
})
Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year', 'IAS', 'SUA', 'VA'],
    num_rows: 0
})


Filter (num_proc=20):   0%|          | 0/4111303 [00:00<?, ? examples/s]

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year', 'IAS', 'SUA', 'VA'],
    num_rows: 0
})


KeyboardInterrupt: 